<a href="https://colab.research.google.com/github/KiselevaStacy/python-ai--/blob/main/notebooks/week2b_read_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Week 2: Data Analysis — Чтение и проверка данных о породах собак

**Цель**: Научиться читать CSV-файлы из вашего репозитория GitHub в Google Colab и выполнять базовую проверку данных о породах собак с помощью pandas.

**Данные:**
- `dog_breeds.csv` — информация о породах собак (страна происхождения, рост, масса, продолжительность жизни)

**Что мы делаем:**
1. Клонируем ваш репозиторий GitHub в Colab
2. Читаем CSV-файл с данными о собаках в pandas DataFrame
3. Очищаем и анализируем структуру данных
4. Проверяем основные статистики (рост, масса, продолжительность жизни)

## 🐱 [1] Клонируем репозиторий курса в Colab

In [1]:
# 🐶 Шаг 1. Клонируем ваш репозиторий в Colab

import os

if not os.path.exists("python-ai--"):
    !git clone -q https://github.com/KiselevaStacy/python-ai--.git

%cd python-ai--

print("✅ Репозиторий готов, работаем внутри папки python-ai--")

/content/python-ai--
✅ Репозиторий готов, работаем внутри папки python-ai--


## 📥 [2A] Простое чтение CSV-файлов в pandas

Сначала просто прочитаем оба CSV-файла в объекты `DataFrame`, без каких‑либо изменений.

После этого мы узнаем, сколько строк загружено в каждый датасет.

In [10]:
# 🐶 Шаг 2A. Чтение CSV-файла в pandas

import pandas as pd

df_dogs = pd.read_csv("data/dog_breeds.csv")

print("✅ Загружено строк:", len(df_dogs))
print("📋 Столбцы:", ", ".join(df_dogs.columns.tolist()))
print("\n👀 Первые 3 строки (сырые данные):")
print(df_dogs.head(3))

✅ Загружено строк: 2420
📋 Столбцы: dogBreed, dogBreedLabel, countryLabel, height, heightQualifier, mass, massQualifier, lifeExpectancy, lifeExpectancyQualifier

👀 Первые 3 строки (сырые данные):
                                dogBreed      dogBreedLabel countryLabel  \
0  http://www.wikidata.org/entity/Q38418  английский мастиф       Англия   
1  http://www.wikidata.org/entity/Q38418  английский мастиф       Англия   
2  http://www.wikidata.org/entity/Q38418  английский мастиф       Англия   

   height                           heightQualifier   mass  \
0    30.0  http://www.wikidata.org/entity/Q10585806  230.0   
1    27.5  http://www.wikidata.org/entity/Q10585806  230.0   
2    30.0  http://www.wikidata.org/entity/Q10585806  170.0   

                              massQualifier  lifeExpectancy  \
0  http://www.wikidata.org/entity/Q10578722             NaN   
1  http://www.wikidata.org/entity/Q10578722             NaN   
2  http://www.wikidata.org/entity/Q10578722             NaN   

## 🧹 [2B] Очистка и переименование столбцов

В исходном CSV-файле есть **технические столбцы**, которые полезны для Викиданных, но мешают простому анализу:

- Столбец `dogBreed` с URL (ссылкой на объект Wikidata) — нам не нужна ссылка, нам достаточно названия породы.
- Столбцы `dogBreedLabel`, `countryLabel` содержат читаемые подписи (название породы, страна происхождения).

В этом шаге мы:
- удалим столбец с URL Wikidata (`dogBreed`);
- переименуем `dogBreedLabel → dogBreed`, `countryLabel → country`;
- приведём числовые столбцы (`height`, `mass`, `lifeExpectancy`) к числовому типу.

При приведении к числам мы используем:

- `pd.to_numeric(..., errors="coerce")` — преобразует значения в числа, некорректные значения превращает в `NaN`;
- `fillna(0)` — заменяет пропущенные значения (`NaN`) на 0 (для `height` и `mass`);
- `fillna(df["lifeExpectancy"].median())` — для продолжительности жизни логичнее заполнить медианой, а не нулём.

> ⚠️ **Важно:** удаление технических столбцов и переименование `*Label` делает таблицу удобной для дальнейшего анализа пород собак.

In [11]:
# 🧹 Шаг 2B. Универсальная очистка данных

# 1) Удаляем ВСЕ технические столбцы (URL и квалификаторы)
technical_cols = [
    "dogBreed", "breed",  # URL породы
    "heightQualifier", "massQualifier", "lifeExpectancyQualifier",
    "heightUnit", "massUnit", "lifeExpectancyUnit"  # дополнительные технические столбцы
]
cols_to_drop = [col for col in technical_cols if col in df_dogs.columns]

if cols_to_drop:
    df_dogs = df_dogs.drop(columns=cols_to_drop)
    print(f"✅ Удалены технические столбцы: {', '.join(cols_to_drop)}")
else:
    print("ℹ️ Технические столбцы уже удалены")

# 2) Определяем столбец с названием породы (гибкий поиск)
breed_col = None
for col in ["dogBreedLabel", "breedLabel", "Breed", "breed", "name", "label"]:
    if col in df_dogs.columns:
        breed_col = col
        break

if breed_col:
    df_dogs = df_dogs.rename(columns={breed_col: "dogBreed"})
    print(f"✅ Найден и переименован столбец породы: '{breed_col}' → 'dogBreed'")
else:
    print("⚠️ Столбец с названием породы не найден! Добавляем заглушку.")
    df_dogs["dogBreed"] = "Неизвестная порода"

# 3) Определяем столбец со страной
country_col = None
for col in ["countryLabel", "country", "origin", "страна"]:
    if col in df_dogs.columns:
        country_col = col
        break

if country_col and country_col != "country":
    df_dogs = df_dogs.rename(columns={country_col: "country"})
    print(f"✅ Найден и переименован столбец страны: '{country_col}' → 'country'")
elif "country" not in df_dogs.columns:
    print("⚠️ Столбец 'country' не найден, создаём пустой столбец")
    df_dogs["country"] = "Неизвестно"

# 4) Приводим числовые столбцы к корректному типу
for col in ["height", "mass"]:
    if col in df_dogs.columns:
        df_dogs[col] = pd.to_numeric(df_dogs[col], errors="coerce").fillna(0).astype(int)

if "lifeExpectancy" in df_dogs.columns:
    median_life = df_dogs["lifeExpectancy"].median()
    df_dogs["lifeExpectancy"] = pd.to_numeric(
        df_dogs["lifeExpectancy"], errors="coerce"
    ).fillna(median_life).round(1)

# 5) Выводим финальную структуру
print("\n✅ Очистка завершена")
print("📋 Финальные столбцы:", ", ".join(df_dogs.columns.tolist()))
print("\n👀 Пример данных:")
display_cols = [col for col in ["dogBreed", "country", "height", "mass", "lifeExpectancy"] if col in df_dogs.columns]
print(df_dogs.head(3)[display_cols].to_string(index=False))

✅ Удалены технические столбцы: dogBreed, heightQualifier, massQualifier, lifeExpectancyQualifier
✅ Найден и переименован столбец породы: 'dogBreedLabel' → 'dogBreed'
✅ Найден и переименован столбец страны: 'countryLabel' → 'country'

✅ Очистка завершена
📋 Финальные столбцы: dogBreed, country, height, mass, lifeExpectancy

👀 Пример данных:
         dogBreed country  height  mass  lifeExpectancy
английский мастиф  Англия      30   230            12.0
английский мастиф  Англия      27   230            12.0
английский мастиф  Англия      30   170            12.0


## 🔍 [3] Обзор данных: структура и первые строки

Сделаем короткий обзор таблицы с породами собак:

- посмотрим размер таблицы (`shape`);
- выведем список столбцов;
- посмотрим первые несколько строк;
- дополнительно посчитаем базовую статистику по ключевым параметрам: рост (`height`), масса (`mass`), продолжительность жизни (`lifeExpectancy`).

Для удобства напишем функцию `show_info(df, name)`, чтобы структурировать вывод информации о данных.

In [12]:
def show_info(df, name, n=5):
    """Краткий обзор DataFrame: имя, размер, список столбцов и первые строки."""
    print(f"\n📊 {name}")
    print("Размер:", df.shape)
    print("Столбцы:", ", ".join(df.columns.tolist()))
    print("\nПервые строки:")
    print(df.head(n))

# 🔍 Шаг 3. Обзор данных о породах собак

show_info(df_dogs, "Породы собак (очищенные данные)")

print("\n📈 Статистика по физическим параметрам:")
print("\n📏 Рост (height, см):")
print(df_dogs["height"].describe().round(1))
print("\n⚖️ Масса (mass, кг):")
print(df_dogs["mass"].describe().round(1))
print("\n⏳ Продолжительность жизни (lifeExpectancy, лет):")
print(df_dogs["lifeExpectancy"].describe().round(1))

print("\n🌍 Топ-10 стран по количеству пород:")
print(df_dogs["country"].value_counts().head(10))


📊 Породы собак (очищенные данные)
Размер: (2420, 5)
Столбцы: dogBreed, country, height, mass, lifeExpectancy

Первые строки:
            dogBreed country  height  mass  lifeExpectancy
0  английский мастиф  Англия      30   230            12.0
1  английский мастиф  Англия      27   230            12.0
2  английский мастиф  Англия      30   170            12.0
3  английский мастиф  Англия      27   170            12.0
4  английский мастиф  Англия      30   160            12.0

📈 Статистика по физическим параметрам:

📏 Рост (height, см):
count    2420.0
mean       47.3
std        22.2
min         0.0
25%        39.0
50%        54.0
75%        62.0
max        90.0
Name: height, dtype: float64

⚖️ Масса (mass, кг):
count    2420.0
mean       17.3
std        20.1
min         0.0
25%         0.0
50%        13.0
75%        30.0
max       230.0
Name: mass, dtype: float64

⏳ Продолжительность жизни (lifeExpectancy, лет):
count    2420.0
mean       12.0
std         0.7
min         8.0
25%       

## ✅ [4] Быстрая проверка и валидация данных

Здесь мы посмотрим:

- сколько **уникальных пород собак** и **стран происхождения** есть в данных;
- **какие страны встречаются чаще всего** (Топ‑5 по числу записей);
- **самые высокие и самые тяжёлые породы** — для выявления возможных выбросов;
- **проверка на аномалии**: нулевые/отрицательные значения роста, массы, продолжительности жизни.

Функция `value_counts()`:
- считает, сколько раз каждое значение встречается в столбце;
- сортирует результаты по убыванию.

Метод `.nunique()` возвращает количество уникальных значений в столбце — полезно для оценки разнообразия данных.

In [13]:
# ✅ Шаг 4. Валидация данных (устойчивая к отсутствию столбцов)

print("🔍 Валидация данных о породах собак")

# 1) Уникальные значения (с проверкой существования столбцов)
print("\n📊 Уникальные значения:")
if "dogBreed" in df_dogs.columns:
    print("Уникальных пород:", df_dogs["dogBreed"].nunique())
else:
    print("⚠️ Столбец 'dogBreed' отсутствует")

if "country" in df_dogs.columns:
    print("Уникальных стран:", df_dogs["country"].nunique())
    print("\n🌍 Топ-5 стран по числу записей:")
    print(df_dogs["country"].value_counts().head())
else:
    print("⚠️ Столбец 'country' отсутствует")

# 2) Проверка аномалий для числовых столбцов
print("\n⚠️ Проверка на аномалии:")
for col in ["height", "mass", "lifeExpectancy"]:
    if col in df_dogs.columns:
        zeros = (df_dogs[col] == 0).sum()
        negatives = (df_dogs[col] < 0).sum()
        print(f"  • {col}: нулевых={zeros}, отрицательных={negatives}")

# 3) Топ по числовым параметрам (если столбцы существуют)
if "height" in df_dogs.columns and "dogBreed" in df_dogs.columns:
    print("\n📏 Топ-5 самых высоких записей (см):")
    cols = ["dogBreed"] + ([col for col in ["country", "height"] if col in df_dogs.columns])
    print(df_dogs.nlargest(5, "height")[cols].to_string(index=False))

if "mass" in df_dogs.columns and "dogBreed" in df_dogs.columns:
    print("\n⚖️ Топ-5 самых тяжёлых записей (кг):")
    cols = ["dogBreed"] + ([col for col in ["country", "mass"] if col in df_dogs.columns])
    print(df_dogs.nlargest(5, "mass")[cols].to_string(index=False))

🔍 Валидация данных о породах собак

📊 Уникальные значения:
Уникальных пород: 586
Уникальных стран: 110

🌍 Топ-5 стран по числу записей:
country
Португалия        244
Франция           202
Германия          191
США               151
Великобритания    142
Name: count, dtype: int64

⚠️ Проверка на аномалии:
  • height: нулевых=310, отрицательных=0
  • mass: нулевых=906, отрицательных=0
  • lifeExpectancy: нулевых=0, отрицательных=0

📏 Топ-5 самых высоких записей (см):
             dogBreed  country  height
         Немецкий дог Германия      90
         Немецкий дог Германия      90
         Немецкий дог Германия      90
         Немецкий дог Германия      90
русская псовая борзая   Россия      85

⚖️ Топ-5 самых тяжёлых записей (кг):
         dogBreed country  mass
английский мастиф  Англия   230
английский мастиф  Англия   230
английский мастиф  Англия   170
английский мастиф  Англия   170
английский мастиф  Англия   160


## 📝 Summary

**Что мы сделали в этом ноутбуке (Week 2):**

- ✅ Клонировали репозиторий [`KiselevaStacy/python-ai--`](https://github.com/KiselevaStacy/python-ai--) в Google Colab
- ✅ Прочитали расширенный файл `data/dog_breeds.csv` с квалификаторами из Викиданных
- ✅ Выполнили комплексную очистку данных:
  - Удалили 4 технических столбца: `dogBreed` (URL), `heightQualifier`, `massQualifier`, `lifeExpectancyQualifier`
  - Переименовали `dogBreedLabel → dogBreed`, `countryLabel → country`
  - Привели числовые столбцы к корректным типам с обработкой пропусков
- ✅ Проанализировали структуру и базовую статистику (рост, масса, продолжительность жизни)
- ✅ Провели валидацию: уникальные породы/страны, проверка аномалий, топ по параметрам

**Особенность данных:**  
Исходный датасет содержал квалификаторы Викиданных (минимум/максимум для роста и массы), что создаёт несколько записей на одну породу. После очистки все измерения сохранены — для получения единого значения на породу потребуется агрегация (среднее/мин/макс) в следующих ноутбуках.

Теперь у нас есть **очищенный датасет**, готовый к углублённому анализу и визуализации.